In [23]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [24]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [25]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [26]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'ESAB',
       'FCFS', 'GEN', 'HUBB', 'IBKR', 'ISEE', 'JJC', 'MGI', 'PEAK', 'PFG',
       'PKI', 'ROCC', 'SBNY', 'SRPT', 'UA', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [27]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ACM', 'ACN', 'ADP', 'AGCO', 'AMCR', 'AMED', 'AMG', 'AMGN', 'ANSS',
       'ARW',
       ...
       'VICR', 'VNDA', 'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WTS', 'WTW',
       'XEL'],
      dtype='object', length=218)


In [28]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'BJ', 'BRBR', 'CARR', 'CDAY', 'CEG', 'CHK',
       'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DCP', 'DDOG', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETH-USD', 'ETRN', 'FISV', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS',
       'GO', 'GXO', 'ISEE', 'JJC', 'KD', 'LCID', 'MGI', 'MP', 'MRNA', 'NARI',
       'NVST', 'NVT', 'OGN', 'OTIS', 'PDD', 'PGNY', 'RIVN', 'ROCC', 'SGOV',
       'SHC', 'SITM', 'SWAV', 'VICI', 'VNT', 'VSCO', 'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [29]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 280


In [30]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-07-13,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['SHV', 'SKYW']",[],"['BCC', 'NVDA']","['BECN', 'BZH', 'FTSM', 'META', 'STRL']","['CCL', 'KDNY', 'NCLH', 'W']","['AVGO', 'BLDR', 'CAR', 'CELH', 'CPRT', 'CWH',...",[]


In [31]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],"['SHV', 'SKYW']",[],"['BCC', 'NVDA']","['BECN', 'BZH', 'FTSM', 'META', 'STRL']","['CCL', 'KDNY', 'NCLH', 'W']","['AVGO', 'BLDR', 'CAR', 'CELH', 'CPRT', 'CWH',...",[]


In [32]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [33]:
import ast

In [34]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 3320
unique symbol count from model picks: 172


In [35]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [36]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['DCP', 'ISEE', 'PKX', 'SRPT']


In [37]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [38]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 280
len(common_symbols): 4
len(symbols_drop): 276


In [39]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 276


In [40]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      27
symbols with same volume:   218
symbols with no close:       57

symbols total before drop:                                        1,581
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   276

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,305    1,500
df Close (df_c) after dropped symbols:      1,305    1,500


In [41]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [42]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [43]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-07-10  116.730003  119.519997  116.559998  118.360001  1889200.0   
2023-07-11  118.300003  118.980003  117.639999  118.480003  1096700.0   
2023-07-12  120.000000  120.489998  119.150002  120.000000  1595400.0   
2023-07-13  120.760002  120.870003  118.989998  119.349998  1229800.0   
2023-07-14  119.540001  120.389999  118.250000  119.750000  1225400.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-07-10  33.700001  34.570000  33.599998  34.139999  3891400.0  ...   
2023-07-11  33.599998  34.869999  33.330002  34.689999  5218400.0  ...   
2023-07-12  35.810001  35.950001  35.000000  35.049999  4288700.0  ...   
2023-07-13  35.700001  37.169998  35.549999  36.689999  5835900.0  ...   
2023-07-14  35.500000  35.520000  34.049999  34.520000  8079500.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-07-10  17.370001  17.610001  16.809999  17.309999  362400.0  26.049999   
2023-07-11  17.450001  17.799999  17.379999  17.730000  278900.0  27.020000   
2023-07-12  18.059999  18.420000  17.680000  17.900000  296700.0  27.480000   
2023-07-13  17.910000  17.910000  17.450001  17.459999  307100.0  27.450001   
2023-07-14  17.420000  17.670000  17.010000  17.650000  376900.0  27.209999   

                                                        
                 High        Low      Close     Volume  
Date                                                    
2023-07-10  26.730000  26.049999  26.690001  1125300.0  
2023-07-11  27.209999  26.780001  27.100000   881300.0  
2023-07-12  27.780001  27.090000  27.180000   880000.0  
2023-07-13  27.639999  27.200001  27.379999   777300.0  
2023-07-14  27.430000  26.920000  27.160000   460600.0  

[5 rows x 6525 columns]

In [44]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABB,ABBV,ABC,ABM,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-10,118.360001,34.139999,18.570000,68.556892,188.610001,31.680000,37.799999,133.015381,192.830002,41.799999,...,32.049999,142.509995,306.160004,72.709999,46.599998,29.559999,25.100000,170.220001,17.309999,26.690001
2023-07-11,118.480003,34.689999,18.799999,69.872231,188.080002,31.780001,38.180000,134.083542,193.220001,42.540001,...,33.270000,143.179993,309.160004,74.120003,51.619999,30.430000,26.250000,170.139999,17.730000,27.100000
2023-07-12,120.000000,35.049999,18.570000,70.500000,189.770004,32.459999,39.279999,133.500000,190.800003,42.540001,...,34.419998,142.649994,315.929993,73.330002,52.150002,31.280001,26.700001,171.860001,17.900000,27.180000
2023-07-13,119.349998,36.689999,18.450001,70.660004,190.539993,34.240002,39.520000,133.589996,191.279999,42.540001,...,35.009998,143.110001,318.470001,74.470001,53.299999,32.000000,27.299999,171.000000,17.459999,27.379999
2023-07-14,119.750000,34.520000,18.139999,70.639999,190.690002,33.619999,39.430000,136.009995,191.419998,42.459999,...,34.180000,142.369995,309.279999,73.150002,52.389999,30.700001,27.120001,172.190002,17.650000,27.160000
